In [4]:
if sequence["five_length"] != 0:
            Nter = SeqRecord(seq = Seq(sequence["nter_elongate"]),
                            id = f"{sequence['seq_id']}-{clustered_sequences}-5utr-{int(len(sequence['nter_elongate']))}",
                            description = "")
            
            nter_db.append(Nter)

        if sequence["three_length"] != 0:
            Cter = SeqRecord(seq = Seq(sequence["cter_elongate"]),
                            id = f"{sequence['seq_id']}-{clustered_sequences}-3utr-{int(len(sequence['cter_elongate']))}",
                            description = "")
            
            cter_db.append(Cter)

{'species': 'Scer_NCBI',
 'seq_id': 'rna-NM_001180415.1',
 'five_length': 18,
 'three_length': 0}